In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mlflow
import optax
import polars as pl
from flax import nnx
from flax_trainer.trainer import Trainer
from sklearn.model_selection import train_test_split

from nn_recsys.encoder import SequentialEncoder
from nn_recsys.evaluator import SequentialEvaluator
from nn_recsys.loader import SequentialLoader
from nn_recsys.loss_fn import cross_entropy_loss
from nn_recsys.model import GRU4Rec

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### READ

In [3]:
dataset_df = (
    pl.read_csv("/workspace/dataset/amazon-m2/sessions_train.csv")
    .filter(pl.col("locale") == "ES")
    .with_columns(
        pl.col("prev_items")
        .str.split(by="'")
        .list.eval(pl.element().filter(pl.arange(0, pl.len()) % 2 == 1))
    )
    .with_columns(
        pl.concat_list(pl.col("prev_items"), pl.col("next_item")).alias("item_ids")
    )
)
display(dataset_df)

train_df, valid_df = train_test_split(
    dataset_df, test_size=0.1, random_state=0, shuffle=True
)

encoder = SequentialEncoder()
encoder.fit(train_df.get_column("item_ids").to_list())

prev_items,next_item,locale,item_ids
list[str],str,str,list[str]
"[""B08MV5B53K"", ""B08MV4RCQR"", ""B08MV5B53K""]","""B012408XPC""","""ES""","[""B08MV5B53K"", ""B08MV4RCQR"", … ""B012408XPC""]"
"[""B07JGW4QWX"", ""B085VCXHXL""]","""B07JFPYN5P""","""ES""","[""B07JGW4QWX"", ""B085VCXHXL"", ""B07JFPYN5P""]"
"[""B08BFQ52PR"", ""B08LVSTZVF"", ""B08BFQ52PR""]","""B08NJP3KT6""","""ES""","[""B08BFQ52PR"", ""B08LVSTZVF"", … ""B08NJP3KT6""]"
"[""B08PPBF9C6"", ""B08PPBF9C6"", … ""B08PPBF9C6""]","""B08PP6BLLK""","""ES""","[""B08PPBF9C6"", ""B08PPBF9C6"", … ""B08PP6BLLK""]"
"[""B0B6W67XCR"", ""B0B712FY2M"", ""B0B6ZYJ3S2""]","""B09SL4MBM2""","""ES""","[""B0B6W67XCR"", ""B0B712FY2M"", … ""B09SL4MBM2""]"
…,…,…,…
"[""B08LR8CH7S"", ""B00CWKFYES""]","""B00C2U6794""","""ES""","[""B08LR8CH7S"", ""B00CWKFYES"", ""B00C2U6794""]"
"[""B08KH2MTSS"", ""B08KJP91X2""]","""B0BGYMJM5S""","""ES""","[""B08KH2MTSS"", ""B08KJP91X2"", ""B0BGYMJM5S""]"
"[""B09ZV92J5P"", ""B06ZY1MXNG"", ""B07W6NS1VC""]","""B09TDTC96J""","""ES""","[""B09ZV92J5P"", ""B06ZY1MXNG"", … ""B09TDTC96J""]"


In [4]:
batch_size = 512

loader = SequentialLoader(
    sequences=train_df.get_column("item_ids").to_list(),
    encoder=encoder,
    batch_size=batch_size,
    rngs=nnx.Rngs(0),
)

evaluator = SequentialEvaluator(
    sequences=valid_df.get_column("item_ids").to_list(),
    encoder=encoder,
    batch_size=batch_size,
)

### 学習

In [5]:
model = GRU4Rec(
    item_num=encoder.item_num + 1,
    embed_dim=30,
    gru_layer_dims=[30],
    ff_layer_dims=[30],
    output_layer_dim=encoder.item_num + 1,
    rngs=nnx.Rngs(0),
    max_batch_size=batch_size,
)

In [6]:
mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("GRU4Rec")

with mlflow.start_run() as run:
    trainer = Trainer(
        model=model,
        optimizer=optax.adamw(learning_rate=0.001, weight_decay=0.001),
        train_loader=loader,
        loss_fn=cross_entropy_loss,
        valid_evaluator=evaluator,
        early_stopping_patience=10,
        epoch_num=32,
        active_run=run,
    )
    trainer = trainer.fit()

100%|██████████| 113/113 [00:04<00:00, 23.84it/s]


[VALID 000]: loss=10.621649742126465, metrics={'hit_10': 0.0002447680744808167, 'cross_entropy': 10.621649742126465}


100%|██████████| 113/113 [00:04<00:00, 25.46it/s]


[VALID 001]: loss=9.517483711242676, metrics={'hit_10': 0.050758782774209976, 'cross_entropy': 9.517483711242676}


100%|██████████| 113/113 [00:04<00:00, 24.80it/s]


...

[VALID 017]: loss=7.873661041259766, metrics={'hit_10': 0.5046811699867249, 'cross_entropy': 7.873661041259766}


100%|██████████| 113/113 [00:04<00:00, 25.41it/s]


[VALID 018]: loss=7.906472682952881, metrics={'hit_10': 0.5063333511352539, 'cross_entropy': 7.906472682952881}


100%|██████████| 113/113 [00:04<00:00, 24.33it/s]


[VALID 019]: loss=7.932363033294678, metrics={'hit_10': 0.5087810754776001, 'cross_entropy': 7.932363033294678}
🏃 View run unique-koi-969 at: http://localhost:8080/#/experiments/756717060270598937/runs/e6a5d809a8ff4f3ebba70b75d481ca49
🧪 View experiment at: http://localhost:8080/#/experiments/756717060270598937
